# ARTEMIS Debate Demo

This notebook demonstrates a complete end-to-end debate using ARTEMIS, including:
- Setting up debate agents
- Running a multi-round debate
- Viewing the transcript and verdict
- Exporting audit logs (JSON, Markdown, HTML)

In [ ]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

import os
print("API Keys configured:")
print(f"  OpenAI: {'✓' if os.environ.get('OPENAI_API_KEY') else '✗'}")
print(f"  Anthropic: {'✓' if os.environ.get('ANTHROPIC_API_KEY') else '✗'}")
print(f"  Google AI Studio: {'✓' if os.environ.get('GOOGLE_API_KEY') else '✗'}")
print(f"  Vertex AI: {'✓' if os.environ.get('GOOGLE_CLOUD_PROJECT') else '✗'}")
print(f"  DeepSeek: {'✓' if os.environ.get('DEEPSEEK_API_KEY') else '✗'}")

## 1. Setup Debate Components

In [ ]:
from artemis.core import Debate, Agent
from artemis.core.jury import JuryPanel
from artemis.core.types import JuryPerspective, JurorConfig

# Choose model based on available API keys
if os.environ.get('GOOGLE_CLOUD_PROJECT'):
    MODEL = "gemini-2.0-flash"
    print(f"Using Vertex AI: {MODEL}")
elif os.environ.get('OPENAI_API_KEY'):
    MODEL = "gpt-4o-mini"  # Cost-effective for demos
    print(f"Using OpenAI: {MODEL}")
else:
    raise ValueError("No API key configured! Set OPENAI_API_KEY or GOOGLE_CLOUD_PROJECT")

In [ ]:
# Create debate agents with different positions
pro_agent = Agent(
    name="Advocate",
    role="Proponent",
    position="in favor of remote work as the default",
    model=MODEL,
    persona="You are a forward-thinking workplace strategist who believes in flexibility and work-life balance."
)

con_agent = Agent(
    name="Traditionalist",
    role="Opponent",
    position="against remote work as the default",
    model=MODEL,
    persona="You are a pragmatic business leader who values in-person collaboration and company culture."
)

print(f"Created agents: {pro_agent.name} vs {con_agent.name}")

In [ ]:
# Create a diverse jury panel
jury = JuryPanel(
    evaluators=3,
    model=MODEL,
    consensus_threshold=0.7,
)

print(f"Jury panel: {len(jury)} jurors")
print(f"Perspectives: {[j.perspective.value for j in jury.jurors]}")

## 2. Run the Debate

In [ ]:
# Create and configure the debate
debate = Debate(
    topic="Should remote work be the default for knowledge workers?",
    agents=[pro_agent, con_agent],
    rounds=2,  # Keep short for demo
    jury=jury,
)

print(f"Debate topic: {debate.topic}")
print(f"Rounds: {debate.rounds}")
print(f"Agents: {[a.name for a in debate.agents]}")

In [ ]:
# Run the debate (this may take a minute)
print("Running debate...")
result = await debate.run()
print(f"\n✓ Debate complete! {len(result.transcript)} turns recorded.")

## 3. View Results

In [ ]:
# Display verdict
from IPython.display import Markdown, HTML

verdict_md = f"""
## Verdict

| Metric | Value |
|--------|-------|
| **Winner** | {result.verdict.decision} |
| **Confidence** | {result.verdict.confidence:.0%} |
| **Unanimous** | {'Yes' if result.verdict.unanimous else 'No'} |

### Reasoning
{result.verdict.reasoning}

### Score Breakdown
"""
for agent, score in result.verdict.score_breakdown.items():
    verdict_md += f"- **{agent}**: {score:.2f}\n"

display(Markdown(verdict_md))

In [ ]:
# Display transcript
transcript_md = "## Debate Transcript\n\n"

for turn in result.transcript:
    transcript_md += f"### Round {turn.round} - {turn.agent} ({turn.argument.level.value})\n\n"
    transcript_md += f"{turn.argument.content}\n\n"
    
    if turn.argument.evidence:
        transcript_md += "**Evidence:**\n"
        for ev in turn.argument.evidence:
            transcript_md += f"- [{ev.type}] {ev.content[:100]}...\n"
        transcript_md += "\n"
    
    transcript_md += "---\n\n"

display(Markdown(transcript_md))

## 4. Export Audit Logs

In [ ]:
from pathlib import Path
from artemis.utils.audit import export_debate_audit, AuditLog

# Create output directory
output_dir = Path("./audit_output")
output_dir.mkdir(exist_ok=True)

# Export in all formats
paths = export_debate_audit(
    result,
    output_dir=output_dir,
    formats=["json", "markdown", "html"]
)

print("Exported audit logs:")
for fmt, path in paths.items():
    print(f"  {fmt.upper()}: {path}")

In [ ]:
# Display JSON structure
import json

with open(paths['json']) as f:
    audit_json = json.load(f)

print("JSON Audit Log Structure:")
print(f"  - debate_id: {audit_json['debate_id'][:8]}...")
print(f"  - topic: {audit_json['topic'][:50]}...")
print(f"  - entries: {len(audit_json['entries'])} events")
print(f"  - verdict: {audit_json['verdict']['decision']}")
print(f"  - metadata: {list(audit_json['metadata'].keys())}")

In [ ]:
# Display HTML report in notebook
with open(paths['html']) as f:
    html_content = f.read()

# Show HTML report (in iframe to contain styles)
from IPython.display import IFrame
display(IFrame(src=str(paths['html']), width='100%', height=600))

In [ ]:
# Show Markdown report
with open(paths['markdown']) as f:
    md_content = f.read()

display(Markdown(md_content))

## 5. Analyze Individual Arguments

In [ ]:
# Show evaluation scores for each turn
import pandas as pd

eval_data = []
for turn in result.transcript:
    if turn.evaluation:
        eval_data.append({
            'Round': turn.round,
            'Agent': turn.agent,
            'Level': turn.argument.level.value,
            'Total Score': turn.evaluation.total_score,
            'Causal Score': turn.evaluation.causal_score,
            'Evidence Count': len(turn.argument.evidence),
        })

if eval_data:
    df = pd.DataFrame(eval_data)
    display(df)
else:
    print("No evaluation data available.")

In [ ]:
# Summary statistics
print("\nDebate Statistics:")
print(f"  Total turns: {len(result.transcript)}")
print(f"  Rounds completed: {result.metadata.total_rounds}")
print(f"  Safety alerts: {len(result.safety_alerts)}")
print(f"  Final state: {result.final_state.value}")

if result.metadata.model_usage:
    print("\nToken Usage:")
    for model, usage in result.metadata.model_usage.items():
        print(f"  {model}: {usage.get('prompt_tokens', 0)} prompt, {usage.get('completion_tokens', 0)} completion")